# Gauge transformation in the GPU including the z dependence

### Set parameters and enviroment variables

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
# N = 128   
N = 16 
tau_sim = 1     
DTS = 4     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

# TODO: Run more events
nevents = 1

In [2]:
import os

os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.lc_gauge as lc

/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using CUDA
Using SU(3)
Using double precision


### We define the simulation routine

In [3]:
from tqdm import tqdm

# Simulation rutine
def simulate():
    output = {}
    
    # Derived parameters
    a = L/N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)
    
    #TODO: only to compare with CPU, remove after
    mv.set_seed(24)
    
    # We create the object simulation
    s = core.Simulation(N, DT, g)

    # We initilize the Glasma fields
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)
    
    # We create the necessary objects for the gauge transformations
    nplus = maxt//DTS
    lc = lc.LCGaugeTransf(s, nplus)
    
    # We create the object where we will store the transformed field
    uplus_lc = su.GROUP_TYPE(np.zeros((maxt//DTS-1, N*N, su.GROUP_ELEMENTS)))
    
    
    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            if t%DTS == 0:
                
                xplus = t//DTS
                lc.evolve_lc(xplus, a)

                if xplus!= 0:
                    uplus_lc[xplus-1] = lc.up_lc.copy()

    if use_cuda:
        cuda.current_context().deallocations.clear()
        
    
    # We write the transformed fields in a dictionary
    output["nplus"] = nplus
    output["uplus_lc"] = uplus_lc
    
    return output

In [4]:
output = simulate()

/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (112) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (112) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (112) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (112) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWa

Kernel xi: 228 d:  0 did not reach goal. check:  0.000000
Kernel xi: 231 d:  0 did not reach goal. check:  0.000983
Kernel xi: 232 d:  0 did not reach goal. check:  0.000000
Kernel xi: 233 d:  0 did not reach goal. check:  0.934217
Kernel xi: 234 d:  0 did not reach goal. check:  0.000000
Kernel xi: 236 d:  0 did not reach goal. check:  0.000000
Kernel xi: 237 d:  0 did not reach goal. check:  0.843181
Kernel xi: 238 d:  0 did not reach goal. check:  2.358576
Kernel xi: 241 d:  0 did not reach goal. check:  1.157817
Kernel xi: 242 d:  0 did not reach goal. check:  1.086507
Kernel xi: 243 d:  0 did not reach goal. check:  1.773258
Kernel xi: 247 d:  0 did not reach goal. check:  0.000000
Kernel xi: 248 d:  0 did not reach goal. check:  0.000000
Kernel xi: 249 d:  0 did not reach goal. check:  0.096570
Kernel xi: 253 d:  0 did not reach goal. check:  0.636997
Kernel xi: 254 d:  0 did not reach goal. check:  0.416994
Kernel xi: 255 d:  0 did not reach goal. check:  0.533396
Kernel xi: 128

/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (112) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
/home2/carlos.lamas/condacurraun/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (112) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home2/carlos.lamas/cond

UnboundLocalError: local variable 'lc' referenced before assignment